### Migrating from Pydantic V1.x to V2.x

#### Introduction

Pydantic 2.0 was released in June 2023.

This brought along several syntactic changes and enfrom the previous 1.x versions.

This video explores some of the main changes and how you can migrate your existing 1.x based Pydantic code to 2.x.

A full migration guide is provided by Pydantic and is available here:

[https://docs.pydantic.dev/latest/migration/](https://docs.pydantic.dev/latest/migration/)

Pydantic provides a migration tool `bump-pydantic` that can help migrate an existing V1 codebase to V2. To be honest, I do not use it - I much prefer taking the time to review my code and make sure I am not only migrating it to V2 but double-checking everything and formatting things the way **I** want. (I don't usually use code generators for the same reasons), However feel free to try it and see how well it works for you.

For this video, your Python environment will need the following libraries installed:
- `pydantic`
- `bump-pydantic` (if you want to try the migration tool on your own)
- `pyhumps` (for camel and snake case conversions - feel free to use whatever you prefer)

#### Deserializing Data

The way we "load" (deserialize) Json or dictionary data into a Pydantic model has changed from using methods such as `parse_obj()`, `parse_raw()` to `model_validate()` and `model_validate_json()`.

We still have access to the V1 BaseModel. You can always use this to make your migrations simpler by using the older V1 BaseModel in conjunction with the new V2 BaseModel.

Here, we'll use it to see the differences between the two and how to re-write V1 BaseModels into V2 BaseModels.

In [1]:
import json
import humps
from pydantic import BaseModel
from pydantic.v1 import BaseModel as BaseModelV1

In [2]:
class CarV1(BaseModelV1):
    manufacturer: str
    model: str
    year: int
    notes: str | None = None        

In [3]:
class Car(BaseModel):
    manufacturer: str
    model: str
    year: int
    notes: str | None = None    

Now let's look at how to deserialize data from JSON and a plain `dict`.

In [4]:
data = {
    "manufacturer": "Citroen",
    "model": "2CV",
    "year": 1981,
    "notes": "Famous James Bond car from 'For Your Eyes Only'"
}

In [5]:
data_json = json.dumps(data)

print(data_json)

{"manufacturer": "Citroen", "model": "2CV", "year": 1981, "notes": "Famous James Bond car from 'For Your Eyes Only'"}


The V1 way of doing it:

In [6]:
CarV1.parse_obj(data)

CarV1(manufacturer='Citroen', model='2CV', year=1981, notes="Famous James Bond car from 'For Your Eyes Only'")

In [7]:
CarV1.parse_raw(data_json)

CarV1(manufacturer='Citroen', model='2CV', year=1981, notes="Famous James Bond car from 'For Your Eyes Only'")

And the V2 way of doing it:

In [8]:
Car.model_validate(data)

Car(manufacturer='Citroen', model='2CV', year=1981, notes="Famous James Bond car from 'For Your Eyes Only'")

In [9]:
Car.model_validate_json(data_json)

Car(manufacturer='Citroen', model='2CV', year=1981, notes="Famous James Bond car from 'For Your Eyes Only'")

#### Serializing Data

Simiarly, the methods to serialize a Pydantic model to JSON or a plain `dict` have been renamed from `json()` and `dict()` to `model_dump_json()` and `model_dump()` respectively:

In [10]:
cv = CarV1.parse_obj(data)

In [11]:
cv.dict()

{'manufacturer': 'Citroen',
 'model': '2CV',
 'year': 1981,
 'notes': "Famous James Bond car from 'For Your Eyes Only'"}

In [12]:
cv.json()

'{"manufacturer": "Citroen", "model": "2CV", "year": 1981, "notes": "Famous James Bond car from \'For Your Eyes Only\'"}'

In [13]:
cv = Car.model_validate(data)

In [14]:
cv.model_dump()

{'manufacturer': 'Citroen',
 'model': '2CV',
 'year': 1981,
 'notes': "Famous James Bond car from 'For Your Eyes Only'"}

In [15]:
cv.model_dump_json()

'{"manufacturer":"Citroen","model":"2CV","year":1981,"notes":"Famous James Bond car from \'For Your Eyes Only\'"}'

The way to use aliases when dumping the data remains the same (albeit with the different methods to serialize the data).

In [16]:
from pydantic import Field

class Person(BaseModel):
    first_name: str = Field(alias="firstName")
    last_name: str = Field(alias="lastName")

In [17]:
p = Person(firstName='Isaac', lastName='Newton')
p

Person(first_name='Isaac', last_name='Newton')

In [18]:
p.model_dump()

{'first_name': 'Isaac', 'last_name': 'Newton'}

In [19]:
p.model_dump(by_alias=True)

{'firstName': 'Isaac', 'lastName': 'Newton'}

In [20]:
p.model_dump_json()

'{"first_name":"Isaac","last_name":"Newton"}'

In [21]:
p.model_dump_json(by_alias=True)

'{"firstName":"Isaac","lastName":"Newton"}'

#### Configuring the Model

In V1 we could customize our model in various ways.

For example, we may have wanted to always use automatically generated aliases for our fields (to account for Python naming conventions, using snake case, and JSON naming conventions, using camel-case). As well as the ability to deserialize data using either the alias or the actual Python field name.

In V1, we would have done it this way:

In [22]:
class Person(BaseModelV1):
    first_name: str
    last_name: str

    class Config:
        allow_population_by_field_name = True
        alias_generator = humps.camelize
        extra = "ignore"

In [23]:
Person(first_name="Isaac", lastName="Newton", year=1643)

Person(first_name='Isaac', last_name='Newton')

In V2, a special type of dictionary (`TypedDict`) is defined inside the class with the various configurations (whose names have also potentially changed, or been removed, with extras added - again, see the link I gave above for a definitive list).

In [24]:
from pydantic import ConfigDict

In [25]:
class Person(BaseModel):
    first_name: str
    last_name: str

    model_config = ConfigDict(
        populate_by_name=True,
        alias_generator=humps.camelize,
        extra="ignore"
    )

In [26]:
p = Person(first_name="Isaac", lastName="Newton", year=1643)
p

Person(first_name='Isaac', last_name='Newton')

And serializing the data using either field name or the alias works just as we just saw:

In [27]:
p.model_dump()

{'first_name': 'Isaac', 'last_name': 'Newton'}

In [28]:
p.model_dump(by_alias=True)

{'firstName': 'Isaac', 'lastName': 'Newton'}

In [29]:
p.model_dump_json()

'{"first_name":"Isaac","last_name":"Newton"}'

In [30]:
p.model_dump_json(by_alias=True)

'{"firstName":"Isaac","lastName":"Newton"}'

#### More Complex Field Types

The way in which we define more complex field types has also changed slightly.

Let's look at a V1 model first:

In [31]:
from pydantic.v1 import conint

class Person(BaseModelV1):
    first_name: str
    last_name: str
    dob: conint(gt=0, le=3001)

    class Config:
        allow_population_by_field_name = True
        alias_generator = humps.camelize
        extra = "ignore"
    

In [32]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=1643)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

first_name='Isaac' last_name='Newton' dob=1643


In [33]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=-10)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic.v1.error_wrappers.ValidationError'> 1 validation error for Person
dob
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)


We have several alternatives in Pydantic V2.

Let's say we want constrain the `dob` integer to be positive only (and not put an upper bound on it).

The simplest is to use the Pydantic built-in type `PositiveInt`:

In [34]:
from pydantic import PositiveInt

class Person(BaseModel):
    first_name: str
    last_name: str
    dob: PositiveInt

    model_config = ConfigDict(
        populate_by_name=True,
        alias_generator=humps.camelize,
        extra="ignore"
    )

In [35]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=1643)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

first_name='Isaac' last_name='Newton' dob=1643


In [36]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=0)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Person
dob
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.4/v/greater_than


If we really want to constrain the integer with lower **and** upper bounds, we could technicallty still use `conint`. However this has been deprecated and will be removed from Pydantic at some point (v3 according to the docs), reason being that this approach does not work well with static type checkers (as you may already know if you ever tried running a static type checker on Pydantic V1 models).

Instead, we should use an `Annotated` type, from Python's standard typing system, and customize it using Pydantic's `Field` object:

In [37]:
from typing import Annotated

In [38]:
class Person(BaseModel):
    first_name: str
    last_name: str
    dob: Annotated[int, Field(gt=0, le=3000)]

    model_config = ConfigDict(
        populate_by_name=True,
        alias_generator=humps.camelize,
        extra="ignore"
    )

In [39]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=1643)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

first_name='Isaac' last_name='Newton' dob=1643


In [40]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=0)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Person
dob
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.4/v/greater_than


In [41]:
try:
   p =  Person(first_name='Isaac', last_name='Newton', dob=4000)
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Person
dob
  Input should be less than or equal to 3000 [type=less_than_equal, input_value=4000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.4/v/less_than_equal


There are however plenty of built-in Pydantic types, which will probably cover most of your needs - however the `Annotated` option is available when you need it.

You can find a full list of Pydantic types here:

[https://docs.pydantic.dev/latest/api/types/](https://docs.pydantic.dev/latest/api/types/)

`conlist` for constrained lists is still valid for V2:

In [42]:
from pydantic import conlist

class Test(BaseModel):
    items: conlist(item_type=int, min_length=1, max_length=5)

In [43]:
try:
    p = Test(items=[1, 2])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

items=[1, 2]


In [44]:
try:
    p = Test(items=[])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
items
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.4/v/too_short


In [45]:
try:
    p = Test(items=['a'])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
items.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/int_parsing


However, you could also use an `Annotated` type if you prefer:

In [46]:
class Test(BaseModel):
    items: Annotated[list[int], Field(min_length=1, max_length=5)]

In [47]:
try:
    p = Test(items=[1, 2])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

items=[1, 2]


In [48]:
try:
    p = Test(items=[])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
items
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.4/v/too_short


In [49]:
try:
    p = Test(items=['a'])
except Exception as ex:
    print(type(ex), ex)
else:
    print(p)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
items.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/int_parsing


#### Custom Validators

If Python's type hinting and Pydantic's custom types are not sufficient for your validation, then we can make use of custom validators.

For example, suppose we want a validator that checks that some string field always starts with a hash (`#`).

In Pydantic V1, we could have done it this way:

In [50]:
from pydantic.v1 import validator

In [51]:
class Test(BaseModelV1):
    hash_tag: str

    @validator("hash_tag")
    def validate_name(cls, value):
        if not value.startswith('#'):
            raise ValueError("Hash tag must start with a #")
        return value

In [52]:
Test(hash_tag="#python")

Test(hash_tag='#python')

In [53]:
try:
    Test(hash_tag="python")
except Exception as ex:
    print(type(ex), ex)

<class 'pydantic.v1.error_wrappers.ValidationError'> 1 validation error for Test
hash_tag
  Hash tag must start with a # (type=value_error)


In Pydantic V2, the `@validator` decorator has been deprecated, and instead you should use the new `@field_validator` decorator:

In [54]:
from pydantic import field_validator

class Test(BaseModel):
    hash_tag: str

    @field_validator("hash_tag")
    def validate_name(cls, value):
        if not value.startswith('#'):
            raise ValueError("Hash tag must start with a #")
        return value

In [55]:
Test(hash_tag="#python")

Test(hash_tag='#python')

In [56]:
try:
    Test(hash_tag="python")
except Exception as ex:
    print(type(ex), ex)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
hash_tag
  Value error, Hash tag must start with a # [type=value_error, input_value='python', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


Note that this validator will, by default, run **after** Pydantic has run its own validation to ensure the field is a string.

To see this, we could use this example (yes, we could just use Pydantic's built-in functionality to ensure we have a positive integer, but I just want to illustrate the point that our custom validator runs after Pydantic has had first crack at the validation):

In [57]:
class Circle(BaseModel):
    radius: int

    @field_validator("radius")
    def validate_radius(cls, value):
        print(f"Validating radius: {value=}, {type(value)=}")
        if value <= 0:
            raise ValueError("radius needs to be a positive integer")
        return value

In [58]:
Circle(radius="100")

Validating radius: value=100, type(value)=<class 'int'>


Circle(radius=100)

As you can see, our validation function received an `int`, not the `str` we supplied when created an instance of our model.

Just as in V1, there's a lot more you can do with custom validators in V2, beyond just the new syntax.

One thing that you'll find in V2, is more reliance on `Annotated` types.

This holds true for validators as well.

Although we used the `@field_validator` decorator above, we could also just use an `Annonated` type.

Using this approach we can define a validator for a type in general, rather than just a single field in a model - a good way to get reuse out of our validators.

Pydantic V2 provides a few ways to define **when** the validator runs, with options `BeforeValidator`, `AfterValidator`, `WrapValidator` and `PlainValidator`.

In [59]:
from pydantic import AfterValidator, BeforeValidator, PlainValidator, WrapValidator

The way these validators work are explained in detail [here](https://docs.pydantic.dev/latest/concepts/validators/), but simplistically we have this kind of behavior for each one:

- `AfterValidator`: runs after Pydantic's internal parsing and validation (so if your field should be an `int`, then thie after validator(s) will get called once Pydantic has parsed the field value to an `int` - makes writing your validation functions easier, since you can now rely on a precise type for the input value.
- `BeforeValidator`: runs before Pydantic's internal parsing and validation. For example if your field shoudl be of type `int` but a string was passed, your validator function will run before Pydantic has a chance to try to coerce that string into an `int`.
- `PlainValidator`: this works similarly to `BeforeValidator`, but instead of Pydantic continuing to try to parse the rest of the data (and report back on all the parsing errors found), it stops the validation process immediately.
- `WrapValidator`: this is the most versatile (and most complex validator that can run validation code **both** before and after Pydantic's own or your custom validators.

The things with all these validators is you can use multiple of them, and Pydantic will execute each of your validators in a specific order based on whether they are after, before or warp validators. We'll take a look at a simple example of this in a bit.

Let's rework our `Test` model example using this approach - in this case we'll use an `AfterValidator` since we want our hash tag field to definitely be a string before we check that it starts with a `#`.

In [60]:
def validate_hash_tag(value: str) -> str:
    if not value.startswith('#'):
        raise ValueError("Hash tag must start with a #")
    return value

In [61]:
HashTagType = Annotated[str, AfterValidator(validate_hash_tag)]

In [62]:
class Test(BaseModel):
    hash_tag: HashTagType

In [63]:
Test(hash_tag="#python")

Test(hash_tag='#python')

In [64]:
try:
    Test(hash_tag="python")
except Exception as ex:
    print(type(ex), ex)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Test
hash_tag
  Value error, Hash tag must start with a # [type=value_error, input_value='python', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


Now let's look at the difference between the before and after validators with our `Circle` model.

In [65]:
def validate_radius(value):
    print(f"Validating radius: {value=}, {type(value)=}")
    if value <= 0:
        raise ValueError("radius needs to be a positive integer")
    return value

Let's use the `AfterValidator` first:

In [66]:
RadiusType = Annotated[int, AfterValidator(validate_radius)]

In [67]:
class Circle(BaseModel):
    radius: RadiusType

In [68]:
Circle(radius="100")

Validating radius: value=100, type(value)=<class 'int'>


Circle(radius=100)

As you can see, our validation function received an `int`.

Now, let's make if a `BeforeValidator`:

In [69]:
RadiusType = Annotated[int, BeforeValidator(validate_radius)]

class Circle(BaseModel):
    radius: RadiusType

In [70]:
Circle(radius=10)

Validating radius: value=10, type(value)=<class 'int'>


Circle(radius=10)

In [71]:
try:
    Circle(radius="100")
except Exception as ex:
    print(type(ex), ex)

Validating radius: value='100', type(value)=<class 'str'>
<class 'TypeError'> '<=' not supported between instances of 'str' and 'int'


One thing to note that is different with V2, is that a `TypeError` no longer gets "transformed" into a `ValidationError` - so watch out for that if you relied on this in the past when using V1.

In [72]:
class Circle(BaseModelV1):
    radius: int

    @validator("radius", pre=True)
    def validate_radius(cls, value):
        print(f"Validating radius: {value=}, {type(value)=}")
        if value <= 0:
            raise ValueError("radius must be a positive integer")
        return value
    

In [73]:
try:
    Circle(radius="100")
except Exception as ex:
    print(type(ex), ex)

Validating radius: value='100', type(value)=<class 'str'>
<class 'pydantic.v1.error_wrappers.ValidationError'> 1 validation error for Circle
radius
  '<=' not supported between instances of 'str' and 'int' (type=type_error)


See? V1 turned that `TypeError` into a model `ValidationError` - no longer the case in V2 as we just saw.

#### Multi-Field Custom Validators

In V1, we have the option to define validators that can "look back" at previous validated fields to validate the current field.

In the video I did on this (in this channel), we used this example:

In [74]:
from enum import Enum

class PolygonType(Enum):
    triangle = 3
    tetragon = 4
    pentagon = 5
    hexagon = 6    

class Polygon(BaseModelV1):
    polygon_type: PolygonType
    vertices: list[tuple[int | float, int | float]]
        
    @validator('vertices')
    @classmethod
    def validate_vertices(cls, value, values):
        # values has access to fields defined "above" itself in the class, and only as
        # long as that field also passed its validation
        print(type(values))
        polygon_type = values.get('polygon_type')
        if polygon_type:
            num_vertices_required = polygon_type.value
            if len(value) != num_vertices_required:
                raise ValueError(
                    f"For a {polygon_type.name}, exactly {polygon_type.value} "
                    "vertices are required."
                )
        return value

Doing this we now have more advanced validation of the vertices list based on the value of `polygon_type`:

In [75]:
Polygon(polygon_type=PolygonType.triangle, vertices = [(1, 1), (2, 2), (3, 3)])

<class 'dict'>


Polygon(polygon_type=<PolygonType.triangle: 3>, vertices=[(1, 1), (2, 2), (3, 3)])

In [76]:
try:
    Polygon(polygon_type=PolygonType.pentagon, vertices=[(1, 1), (2, 2), (3, 3)])
except Exception as ex:
    print(type(ex), ex)

<class 'dict'>
<class 'pydantic.v1.error_wrappers.ValidationError'> 1 validation error for Polygon
vertices
  For a pentagon, exactly 5 vertices are required. (type=value_error)


In V2, we have similar functionality.

Let's look at a field validator first.

Note how the third argument in the validator is a dictionary containing the previous validated fields (previous as determined by the order of the fields in the Pydantic model).

We saw that for V1, the validator had three arguments: `cls`, `value` and `values`.

The same holds for `@field_validator`, but the type of the third argument has changed:
- the first argument is the class (remember that these decorator validators are class methods, not instance methods)
- the second argument, like in V1 is the value being validated
- the third argument is no longer a dict, but an instance of the `ValidationInfo` class

Let's start to rewrite our example in V2:

In [77]:
from pydantic import ValidationInfo

class Polygon(BaseModel):
    polygon_type: PolygonType
    vertices: list[tuple[int | float, int | float]]
        
    @field_validator('vertices')
    @classmethod
    def validate_vertices(cls, value, info: ValidationInfo):
        print(value)        
        print(type(info))
        

In [78]:
Polygon(polygon_type=PolygonType.triangle, vertices = [(1, 1), (2, 2), (3, 3)])

[(1, 1), (2, 2), (3, 3)]
<class 'pydantic_core._pydantic_core.ValidationInfo'>


Polygon(polygon_type=<PolygonType.triangle: 3>, vertices=None)

The docs for the `ValidationInfo` class is located [here](https://docs.pydantic.dev/latest/api/pydantic_core_schema/#pydantic_core.core_schema.ValidationInfo)

In particular we are interested in the `.data` attribute of that object - that will be similar to the `values` dictionary we had in V1:

In [79]:
from pydantic import ValidationInfo

class Polygon(BaseModel):
    polygon_type: PolygonType
    vertices: list[tuple[int | float, int | float]]
        
    @field_validator('vertices')
    @classmethod
    def validate_vertices(cls, value, info: ValidationInfo):
        print(f"{info.data=}")
        

In [80]:
Polygon(polygon_type=PolygonType.triangle, vertices = [(1, 1), (2, 2), (3, 3)])

info.data={'polygon_type': <PolygonType.triangle: 3>}


Polygon(polygon_type=<PolygonType.triangle: 3>, vertices=None)

Given this, we can now rewrite our field validator this way:

In [81]:
class Polygon(BaseModel):
    polygon_type: PolygonType
    vertices: list[tuple[int | float, int | float]]
        
    @field_validator('vertices')
    @classmethod
    def validate_vertices(cls, value, info:ValidationInfo):
        values = info.data
        polygon_type = values.get('polygon_type')
        if polygon_type:
            num_vertices_required = polygon_type.value
            if len(value) != num_vertices_required:
                raise ValueError(
                    f"For a {polygon_type.name}, exactly {polygon_type.value} "
                    "vertices are required."
                )
        return value

And we get the same functionality as before:

In [82]:
Polygon(polygon_type=PolygonType.triangle, vertices = [(1, 1), (2, 2), (3, 3)])

Polygon(polygon_type=<PolygonType.triangle: 3>, vertices=[(1, 1), (2, 2), (3, 3)])

In [83]:
try:
    Polygon(polygon_type=PolygonType.pentagon, vertices=[(1, 1), (2, 2), (3, 3)])
except Exception as ex:
    print(type(ex), ex)

<class 'pydantic_core._pydantic_core.ValidationError'> 1 validation error for Polygon
vertices
  Value error, For a pentagon, exactly 5 vertices are required. [type=value_error, input_value=[(1, 1), (2, 2), (3, 3)], input_type=list]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


#### Generating a Model's JSON Schema

In V1 we could produce the JSON schema for any model:

In [84]:
class Parent(BaseModelV1):
    field_1: str
    field_2: int

class SubClass(Parent):
    field_3: float

In [85]:
print(SubClass.schema_json(indent=2))

{
  "title": "SubClass",
  "type": "object",
  "properties": {
    "field_1": {
      "title": "Field 1",
      "type": "string"
    },
    "field_2": {
      "title": "Field 2",
      "type": "integer"
    },
    "field_3": {
      "title": "Field 3",
      "type": "number"
    }
  },
  "required": [
    "field_1",
    "field_2",
    "field_3"
  ]
}


We can also generate a JSON schema in V2, but the way to do it is slightly different. The `schema_json()` method has been deprecated (still works, but will go away), and we should use the `model_schema_json()` method.

However, unlike `schema_json()` which returns a string, the `model_schema_json()` method returns a dictionary object (makes it easier to inspect and manipulate the schema object in code, without requiring us to deserialize the JSON string back to a dictionary). 

This does mean that if we want the dictionary as a string, we need to serialize it to a string ourselves.


In [86]:
class Parent(BaseModel):
    field_1: str
    field_2: int

class SubClass(Parent):
    field_3: float 

In [87]:
schema = SubClass.model_json_schema()

In [88]:
print(type(schema))

<class 'dict'>


In [89]:
print(schema)

{'properties': {'field_1': {'title': 'Field 1', 'type': 'string'}, 'field_2': {'title': 'Field 2', 'type': 'integer'}, 'field_3': {'title': 'Field 3', 'type': 'number'}}, 'required': ['field_1', 'field_2', 'field_3'], 'title': 'SubClass', 'type': 'object'}


And if we want it as a string:

In [90]:
print(json.dumps(schema, indent=2))

{
  "properties": {
    "field_1": {
      "title": "Field 1",
      "type": "string"
    },
    "field_2": {
      "title": "Field 2",
      "type": "integer"
    },
    "field_3": {
      "title": "Field 3",
      "type": "number"
    }
  },
  "required": [
    "field_1",
    "field_2",
    "field_3"
  ],
  "title": "SubClass",
  "type": "object"
}


#### Conclusion

And that covers what are probably the most common usages of Pydantic, and how to transition from V1 to V2.

In the future, I will create another Pydantic video, like the V1 video I have in this channel, but specific to V2.